In [1]:
# Sources:
# linux laptop
# amazon.de
# apple new
# apple refurbished
# top notebookcheck multimedia laptop
# x-kom
# delkom
# techlord
# allegro
# refurbished
# huawei laptop
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns", None)
raw_laptops_data = pd.read_csv("laptops.csv")
raw_laptops_data.sort_values(by="Brand")

,URL,Laptop Name,Brand,Brand Reliability,CPU,CPU P,GPU,GPU P,RAM,Screen Size,Resolution,System,System S,Weight,Case,Color Cov,Col.Cov Norm,Storage,Temp,New,Price
0,https://www.apple.com/pl/shop/buy-mac/macbook-...,MacBook Pro M3,Apple,0.80%,M3 Pro,52.90,M3 Pro 14-Core GPU,46.15,36,16.0,7720704,MacOS,50,2.14,100%,DCI-P3 99.2%,98.3072,1000,38.1,100,16399
20,https://www.refurbed.pl/p/apple-macbook-pro-20...,MacBook Pro 2019,Apple,0.80%,i9-9980HK,35.40,Radeon Pro 5500M,27.60,64,16.0,5898240,MacOS,50,2.00,100%,Abode 78%,78.0000,1000,47.3,0,5144
15,https://techlord.pl/asus-vivobook-16x-k3605vu-...,ASUS VivoBook 16X,Asus,3.80%,i9-13900H,46.30,RTX 4050,53.10,32,16.0,2304000,Windows,0,1.80,50.00%,sRGB 100%,88.0000,1000,39.7,100,6499
13,NaN,ASUS VivoBook 15,Asus,3.80%,i3-8145U,17.20,MX250,9.10,8,15.6,2073600,Linux Mint,100,1.90,0%,Abode 40.55%,40.5500,256,NaN,100,2200
23,https://techlord.pl/asus-vivobook-16x-k3605vu-...,VivoBook 16X K3605VU,Asus,3.80%,i9-13900H,46.30,RTX 4050,53.10,32,16.0,2304000,NoOS,0,1.80,90%,sRGB 100%,88.0000,1000,NaN,100,5999
3,https://www.x-kom.pl/p/1246865-notebook-laptop...,ASUS Vivobook S15,Asus,3.80%,Qualcomm Snapdragon X Elite,34.60,Snapdragon X Adreno X1-85,10.70,32,15.6,4665600,Windows,0,1.42,100%,AbodeRGB 98.6%,98.6000,1000,43.6,100,5999
2,https://techlord.pl/asus-zenbook-15-um3504da-r...,Asus ZenBook 15,Asus,3.80%,Ryzen™ 7 7735U,38.00,Radeon 680M,21.70,32,15.6,4665600,Windows,0,1.40,100%,Abode 100%,100.0000,1000,NaN,100,5995
18,https://www.martechgroup.com.pl/laptopy/673-la...,G16 7620,Dell,2.40%,i7-12700H,43.10,RTX 3050 Ti,33.50,32,16.0,4096000,Windows,0,2.48,30%,AbodeRGB 70.5%,70.5000,1000,47.6,100,4599
11,https://www.x-kom.pl/p/1223886-notebook-laptop...,Gigabyte AORUS 16X,Gigabyte,4.30%,i7-13650HX,46.00,RTX4060,60.70,32,16.0,4096000,Windows,0,2.30,0%,sRGB 100%,88.0000,1000,41.0,100,6349
12,https://www.x-kom.pl/p/1142918-notebook-laptop...,Gigabyte AORUS 15 BKF,Gigabyte,4.30%,i7-13700H,44.20,RTX4060,60.70,32,15.6,3686400,Windows,0,2.39,0%,DCI-P3 100%,99.1000,1000,NaN,100,6489


In [2]:
system_score = {
    "Windows": 0,
    "MacOS": 0.5,
    "Linux": 1,
    "NoOS": 1,
    "Ubuntu": 1,
    "Linux Mint": 1,
}

laptops_data = pd.DataFrame()


laptops_data["Brand"] = raw_laptops_data["Brand"]

laptops_data["Brand Reliability"] = 1 - raw_laptops_data["Brand Reliability"].str.replace("%", "").astype(float) / 100
laptops_data["Brand Reliability Score"] = MinMaxScaler().fit_transform(
    laptops_data["Brand Reliability"].values.reshape(-1, 1)
)
laptops_data["Brand Reliability Score"] = laptops_data["Brand Reliability Score"].fillna(-1)


laptops_data["CPU"] = raw_laptops_data["CPU"]
laptops_data["CPU Performance"] = raw_laptops_data["CPU P"]
laptops_data["CPU Performance Score"] = MinMaxScaler().fit_transform(
    laptops_data["CPU Performance"].values.reshape(-1, 1)
)

laptops_data["GPU"] = raw_laptops_data["GPU"]
laptops_data["GPU Performance"] = raw_laptops_data["GPU P"]
laptops_data["GPU Performance Score"] = MinMaxScaler().fit_transform(
    laptops_data["GPU Performance"].values.reshape(-1, 1)
)


laptops_data["RAM"] = raw_laptops_data["RAM"]
laptops_data["RAM Score"] = MinMaxScaler().fit_transform(laptops_data["RAM"].values.reshape(-1, 1))


laptops_data["Screen Size"] = raw_laptops_data["Screen Size"]
laptops_data["Screen Size Score"] = MinMaxScaler().fit_transform(laptops_data["Screen Size"].values.reshape(-1, 1))


laptops_data["Resolution"] = raw_laptops_data["Resolution"]
laptops_data["Resolution Score"] = MinMaxScaler().fit_transform(laptops_data["Resolution"].values.reshape(-1, 1))


laptops_data["System"] = raw_laptops_data["System"]
laptops_data["System Score"] = raw_laptops_data["System"].map(system_score)


laptops_data["Weight"] = raw_laptops_data["Weight"]


laptops_data["Weight Score"] = 1 - MinMaxScaler().fit_transform(laptops_data["Weight"].values.reshape(-1, 1))


laptops_data["Case"] = raw_laptops_data["Case"].str.replace("%", "").astype(float)
laptops_data["Case Score"] = MinMaxScaler().fit_transform(laptops_data["Case"].values.reshape(-1, 1))


laptops_data["Color Cov"] = raw_laptops_data["Col.Cov Norm"]
laptops_data["Color Cov Score"] = MinMaxScaler().fit_transform(laptops_data["Color Cov"].values.reshape(-1, 1))


laptops_data["Storage"] = raw_laptops_data["Storage"]
laptops_data["Storage Score"] = MinMaxScaler().fit_transform(laptops_data["Storage"].values.reshape(-1, 1))

laptops_data["Temperature"] = raw_laptops_data["Temp"]
laptops_data["Temperature Score"] = 1 - MinMaxScaler().fit_transform(laptops_data["Temperature"].values.reshape(-1, 1))
laptops_data["Temperature Score"] = laptops_data["Temperature Score"].fillna(-1)


laptops_data["New"] = raw_laptops_data["New"] / 100


laptops_data["Price"] = raw_laptops_data["Price"]
laptops_data

,Brand,Brand Reliability,Brand Reliability Score,CPU,CPU Performance,CPU Performance Score,GPU,GPU Performance,GPU Performance Score,RAM,RAM Score,Screen Size,Screen Size Score,Resolution,Resolution Score,System,System Score,Weight,Weight Score,Case,Case Score,Color Cov,Color Cov Score,Storage,Storage Score,Temperature,Temperature Score,New,Price
0,Apple,0.992,1.000000,M3 Pro,52.90,1.000000,M3 Pro 14-Core GPU,46.15,0.718023,36,0.500000,16.0,0.35,7720704,0.790645,MacOS,0.5,2.14,3.416667e-01,100.0,1.00,98.3072,0.972608,1000,0.426606,38.1,1.000000,1.0,16399
1,LG,0.978,0.600000,i7-1360P,41.10,0.669468,Iris Xe Graphics G7 96EUs,13.50,0.085271,32,0.428571,17.0,0.85,4096000,0.283154,Windows,0.0,1.35,1.000000e+00,100.0,1.00,84.9000,0.755663,2000,1.000000,43.9,0.467890,1.0,6297
2,Asus,0.962,0.142857,Ryzen™ 7 7735U,38.00,0.582633,Radeon 680M,21.70,0.244186,32,0.428571,15.6,0.15,4665600,0.362903,Windows,0.0,1.40,9.583333e-01,100.0,1.00,100.0000,1.000000,1000,0.426606,NaN,-1.000000,1.0,5995
3,Asus,0.962,0.142857,Qualcomm Snapdragon X Elite,34.60,0.487395,Snapdragon X Adreno X1-85,10.70,0.031008,32,0.428571,15.6,0.15,4665600,0.362903,Windows,0.0,1.42,9.416667e-01,100.0,1.00,98.6000,0.977346,1000,0.426606,43.6,0.495413,1.0,5999
4,Huawei,0.977,0.571429,i7-13700H,44.20,0.756303,Iris Xe Graphics G7 96EUs,13.50,0.085271,32,0.428571,16.0,0.35,4233600,0.302419,Windows,0.0,1.99,4.666667e-01,100.0,1.00,74.0000,0.579288,1000,0.426606,43.2,0.532110,1.0,6379
5,System 76,NaN,-1.000000,Ultra 7 155H,42.10,0.697479,Arc 8-Core iGPU,24.70,0.302326,32,0.428571,16.0,0.35,2304000,0.032258,Linux,1.0,1.93,5.166667e-01,100.0,1.00,88.0000,0.805825,1000,0.426606,NaN,-1.000000,1.0,6286
6,Lenovo,0.969,0.342857,Ryzen 5 5600H,37.00,0.554622,RTX 3050 4GB Laptop GPU,32.60,0.455426,16,0.142857,16.0,0.35,4096000,0.283154,Ubuntu,1.0,2.45,8.333333e-02,100.0,1.00,73.0000,0.563107,512,0.146789,39.9,0.834862,1.0,4999
7,Huawei,0.977,0.571429,Ryzen 7 5800H,38.90,0.607843,Radeon RX Vega 8 (Ryzen 4000/5000),15.00,0.114341,16,0.142857,16.0,0.35,4233600,0.302419,Windows,0.0,1.99,4.666667e-01,100.0,1.00,65.0000,0.433657,512,0.146789,40.2,0.807339,1.0,4548
8,Schenker,NaN,-1.000000,i7-1360P,41.10,0.669468,Iris Xe Graphics G7 96EUs,13.50,0.085271,32,0.428571,17.3,1.00,2073600,0.000000,NoOS,1.0,2.25,2.500000e-01,50.0,0.50,79.2000,0.663430,1000,0.426606,39.1,0.908257,1.0,5756
9,Lenovo,0.969,0.342857,i5-14500HX,52.75,0.995798,GeForce RTX 4060,60.70,1.000000,32,0.428571,16.0,0.35,4096000,0.283154,NoOS,1.0,2.50,4.166667e-02,50.0,0.50,70.1000,0.516181,1000,0.426606,43.8,0.477064,1.0,6499


# Ranking

1. CPU
2. RAM
3. New/Used
4. Price
5. Color Coverage
6. Resolution
7. Case
8. System
9. Screen Size
10. Storage
11. Weight
12. Brand


In [11]:
weights = {
    "CPU": 0.20,
    "Case": 0.1,
    "RAM": 0.1,
    "New/Old": 0.1,
    "Resolution": 0.1,
    "Temperature": 0.05,
    "Screen Size": 0.05,
    "Weight": 0.05,
    "GPU": 0.05,
    "Color Cov": 0.05,
    "System": 0.05,
    "Storage": 0.05,
    "Brand": 0.05,
}

print("Weights sum: ", sum(weights.values()))



laptops_data["Score"] = (
    laptops_data["CPU Performance Score"] * weights["CPU"]
    + laptops_data["GPU Performance Score"] * weights["GPU"]
    + laptops_data["RAM Score"] * weights["RAM"]
    + laptops_data["New"] * weights["New/Old"]
    + laptops_data["Color Cov Score"] * weights["Color Cov"]
    + laptops_data["Resolution Score"] * weights["Resolution"]
    + laptops_data["Case Score"] * weights["Case"]
    + laptops_data["System Score"] * weights["System"]
    + laptops_data["Screen Size Score"] * weights["Screen Size"]
    + laptops_data["Temperature Score"] * weights["Temperature"]
    + laptops_data["Storage Score"] * weights["Storage"]
    + laptops_data["Weight Score"] * weights["Weight"]
    + laptops_data["Brand Reliability Score"] * weights["Brand"]
)



laptops_data["Total Score"] = laptops_data["Score"] / laptops_data["Price"] * 10000



laptops_data = laptops_data.round(2)


laptops_data.sort_values(by=["Total Score", "Score"], ascending=False)

Weights sum:  1.0000000000000002


,Brand,Brand Reliability,Brand Reliability Score,CPU,CPU Performance,CPU Performance Score,GPU,GPU Performance,GPU Performance Score,RAM,RAM Score,Screen Size,Screen Size Score,Resolution,Resolution Score,System,System Score,Weight,Weight Score,Case,Case Score,Color Cov,Color Cov Score,Storage,Storage Score,Temperature,Temperature Score,New,Price,Score,Total Score
14,Xiaomi,NaN,-1.00,Ultra 7 155H,42.00,0.69,Arc 8-Core iGPU,24.70,0.30,32,0.43,16.0,0.35,5898240,0.54,Windows,0.0,1.88,0.56,100.0,1.00,89.90,0.84,1000,0.43,40.3,0.80,1.0,4553,0.55,1.21
7,Huawei,0.98,0.57,Ryzen 7 5800H,38.90,0.61,Radeon RX Vega 8 (Ryzen 4000/5000),15.00,0.11,16,0.14,16.0,0.35,4233600,0.30,Windows,0.0,1.99,0.47,100.0,1.00,65.00,0.43,512,0.15,40.2,0.81,1.0,4548,0.51,1.12
6,Lenovo,0.97,0.34,Ryzen 5 5600H,37.00,0.55,RTX 3050 4GB Laptop GPU,32.60,0.46,16,0.14,16.0,0.35,4096000,0.28,Ubuntu,1.0,2.45,0.08,100.0,1.00,73.00,0.56,512,0.15,39.9,0.83,1.0,4999,0.54,1.08
20,Apple,0.99,1.00,i9-9980HK,35.40,0.51,Radeon Pro 5500M,27.60,0.36,64,1.00,16.0,0.35,5898240,0.54,MacOS,0.5,2.00,0.46,100.0,1.00,78.00,0.64,1000,0.43,47.3,0.16,0.0,5144,0.55,1.07
18,Dell,0.98,0.54,i7-12700H,43.10,0.73,RTX 3050 Ti,33.50,0.47,32,0.43,16.0,0.35,4096000,0.28,Windows,0.0,2.48,0.06,30.0,0.30,70.50,0.52,1000,0.43,47.6,0.13,1.0,4599,0.47,1.03
1,LG,0.98,0.60,i7-1360P,41.10,0.67,Iris Xe Graphics G7 96EUs,13.50,0.09,32,0.43,17.0,0.85,4096000,0.28,Windows,0.0,1.35,1.00,100.0,1.00,84.90,0.76,2000,1.00,43.9,0.47,1.0,6297,0.64,1.02
9,Lenovo,0.97,0.34,i5-14500HX,52.75,1.00,GeForce RTX 4060,60.70,1.00,32,0.43,16.0,0.35,4096000,0.28,NoOS,1.0,2.50,0.04,50.0,0.50,70.10,0.52,1000,0.43,43.8,0.48,1.0,6499,0.63,0.97
23,Asus,0.96,0.14,i9-13900H,46.30,0.82,RTX 4050,53.10,0.85,32,0.43,16.0,0.35,2304000,0.03,NoOS,1.0,1.80,0.62,90.0,0.90,88.00,0.81,1000,0.43,NaN,-1.00,1.0,5999,0.56,0.93
19,Lenovo,0.97,0.34,i5-13500HX,43.90,0.75,RTX 4060,60.70,1.00,32,0.43,16.0,0.35,4096000,0.28,NoOS,1.0,2.55,-0.00,50.0,0.50,73.30,0.57,1000,0.43,42.8,0.57,1.0,6449,0.58,0.91
4,Huawei,0.98,0.57,i7-13700H,44.20,0.76,Iris Xe Graphics G7 96EUs,13.50,0.09,32,0.43,16.0,0.35,4233600,0.30,Windows,0.0,1.99,0.47,100.0,1.00,74.00,0.58,1000,0.43,43.2,0.53,1.0,6379,0.58,0.90
